First diff trial

In [1]:
from commonfunctions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
import cv2
import pytesseract

from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.filters import threshold_otsu

In [ ]:
# Function to detect the sheet contour and apply perspective transform
def detect_sheet_contour(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area (largest first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Check if we have enough contours
    if len(contours) > 0:
        # Assume the largest contour is the sheet
        sheet_contour = contours[0]

        # Approximate the sheet's contour to a quadrilateral
        epsilon = 0.02 * cv2.arcLength(sheet_contour, True)
        approx = cv2.approxPolyDP(sheet_contour, epsilon, True)

        # Ensure we have 4 points (a quadrilateral)
        if len(approx) == 4:
            points = approx.reshape(4, 2)
            rect = np.zeros((4, 2), dtype='float32')

            s = points.sum(axis=1)
            rect[0] = points[np.argmin(s)]  # Top-left
            rect[2] = points[np.argmax(s)]  # Bottom-right

            diff = np.diff(points, axis=1)
            rect[1] = points[np.argmin(diff)]  # Top-right
            rect[3] = points[np.argmax(diff)]  # Bottom-left

            # Define the standard rectangle for the aligned sheet (adjust size based on your sheet)
            (width, height) = (500, 200)  # Adjust the size if needed
            dst = np.array([
                [0, 0],
                [width - 1, 0],
                [width - 1, height - 1],
                [0, height - 1]
            ], dtype='float32')

            # Compute the perspective transform matrix
            M = cv2.getPerspectiveTransform(rect, dst)

            # Apply perspective warp
            aligned_sheet = cv2.warpPerspective(image, M, (width, height))

            # Display the aligned sheet
            cv2.imshow('Aligned Sheet', aligned_sheet)
            show_images([aligned_sheet,edges])

            cv2.waitKey(0)
            cv2.destroyAllWindows()

            return aligned_sheet
        else:
            print("Sheet alignment failed: Unable to detect 4 corners.")
            return None
    else:
        print("No contours detected.")
        return None

In [ ]:
# Function to extract the name from the aligned sheet using OCR
def extract_name(aligned_sheet):
    # Define the region where the name is located (you may need to adjust this region)
    name_region = aligned_sheet[50:150, 10:500]  # Modify based on the actual location

    # Convert the name region to grayscale
    name_gray = cv2.cvtColor(name_region, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to isolate the text
    _, thresh_name = cv2.threshold(name_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Use Tesseract to extract the name text
    extracted_name = pytesseract.image_to_string(thresh_name, config='--psm 6')

    print("Extracted Name:", extracted_name.strip())

    # Optionally, display the extracted name region
    cv2.imshow('Name Region', thresh_name)
    show_images([thresh_name])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Main process
def main():
    # Define the path to your image
    file_path = "3\IMG_2032.jpg"  # Replace with your image path
    
    # Load the image
    image = load_image(file_path)

    if image is not None:
        # Detect the sheet contour and get the aligned sheet
        aligned_sheet = detect_sheet_contour(image)

        if aligned_sheet is not None:
            # Extract the name from the aligned sheet
            extract_name(aligned_sheet)
            
# Run the main function
if __name__ == "__main__":
    main()